# Knowledge Drift Experiment
- Knowledge drift appears when time progresses and more data is available to be shared with peers.
- Ideally, every node in the network will constantly receive the same time slice throughout continued use of the network.

In [1]:
from core.network import *
from core.config import config
config["enumerate_nodes"] = True

## Simulating steady network growth
- The worst case for knowledge drift is a constant amount of nodes over a long period of time.
- A growing network will distribute progressively smaller slices over all nodes.

In [2]:
def simulate_steady_growth(net: Network, days: int, growth_rate: float = 1.0):
    dataset = net.get_all_dataset_names()[0]
    net.distribute_series(dataset)
    net.allocate_dataframes(net.start_date, net.current_date + timedelta(days))
    for i in range(days):
        if i % 30 == 29:
            print(f"Iterated {i+1} days with {len(net.nodes)} nodes...")
        if growth_rate <= 1:
            if i % (1 / growth_rate) < 1:
                net.create_nodes(1, 1)
        else:
            if i % growth_rate <= 1:
                net.create_nodes(np.floor(growth_rate) + 1, 1)
            else:
                net.create_nodes(np.floor(growth_rate), 1)
        net.tick()
        net.distribute_series(dataset)
    net.print_statistics(dataset)
    net.print_dataset_intervals(dataset)
    net.print_dataset_distribution(dataset)

### Execute simulation
- `nodes_cnt`: Initial amount of nodes in the network
- `tree_type`: Tree building algorithm for interval distribution
- `growth_rate`: How many nodes per day are added (**TODO**: May be replaced by a schedule/array)

In [5]:
network = create_network(nodes_cnt=5, days_of_data=30, tree_type=TreeType.balanced_ltor)
simulate_steady_growth(network, days=60, growth_rate=0.1)

Iterated 30 days with 8 nodes...
Iterated 60 days with 11 nodes...

Total nodes: 11
Total slices: 91
Minimum amount of ticks observed:  9.00 of 91 ( 9.9 %)
Average amount of ticks observed:  28.73 of 91 ( 31.6 %)
Maximum amount of ticks observed:  55.00 of 91 ( 60.4 %)

Printing intervals of dataset 4-1
[2022-06-11]------------▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒█████████-----------------------------------------[2022-09-09] -> 0
[2022-06-11]------▒▒▒▒▒▒▒▒▒▒▒████████------------------------------------------------------------------[2022-09-09] -> 1
[2022-06-11]------------------------▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████-----------------[2022-09-09] -> 2
[2022-06-11]▒▒▒▒▒▒▒▒▒████████--------------------------------------------------------------------------[2022-09-09] -> 3
[2022-06-11]------------------▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████-------------------------[2022-09-09] -> 4
[2022-06-11]-----------▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████-------------------------------------